In [433]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [434]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [435]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [436]:
from notebooks.optimization_problems.pointing_problem import PointingProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 0.5e9
#requirements.max_throughput = 5e9

problem = PointingProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 400, 'n_offsprings': 20}

settings = {}

# NSGA-II
from pymoo.algorithms.nsga2 import NSGA2
settings['NSGA-II'] = {
    'label': 'nsga2',
    'algorithm': NSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

# NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=25)
#get_visualization("scatter").add(ref_dirs).show()
settings['NSGA-III'] = {
    'label': 'nsga3',
    'algorithm': NSGA3(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

# R-NSGA-II
from pymoo.algorithms.rnsga2 import RNSGA2
ref_points = np.array([
    [-1800e9, 45e3, 2000],   # min -throughput, min energy, d.c. pointing
    [-250e9, 60e3, 100]     # min -throughput, d.c. energy, min pointing
])
settings['RNSGA-II'] = {
    'label': 'rnsga2',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.001,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

# R-NSGA-II wide
from pymoo.algorithms.rnsga2 import RNSGA2
ref_points = np.array([
    [-1800e9, 45e3, 2000],   # min -throughput, min energy, d.c. pointing
    [-250e9, 60e3, 100]     # min -throughput, d.c. energy, min pointing
])
settings['RNSGA-II-e0_01'] = {
    'label': 'rnsga2_e0_01',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.01,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

setting = settings['NSGA-II']
setting = settings['NSGA-III']
#setting = settings['RNSGA-II']
setting = settings['RNSGA-II-e0_01']

In [437]:
termination = setting['termination']
algorithm = setting['algorithm']

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               #save_history=True,
               verbose=True
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     400 |  0.00000E+00 |  3.03008E+07 |      15 |            - |            -
    2 |     420 |  0.00000E+00 |  8.66701E+06 |      18 |  0.015350141 |        nadir
    3 |     440 |  0.00000E+00 |  1.23808E+06 |      18 |  0.00000E+00 |            f
    4 |     460 |  0.00000E+00 |  1.73448E+02 |      18 |  0.00000E+00 |            f
    5 |     480 |  0.00000E+00 |  1.62870E+02 |      18 |  0.011552673 |            f
    6 |     500 |  0.00000E+00 |  1.49585E+02 |      16 |  0.024558816 |            f
    7 |     520 |  0.00000E+00 |  1.38618E+02 |      15 |  0.004828469 |            f
    8 |     540 |  0.00000E+00 |  1.28010E+02 |      15 |  0.024935107 |            f
    9 |     560 |  0.00000E+00 |  1.18963E+02 |      16 |  0.031919201 |            f
   10 |     580 |  0.00000E+00 |  1.12728E+02 |      17 |  0.010698826 |            f
   11 |     600 |  0.00000E+00 |  1.06123E+02 |      1

In [438]:
pickle.dump((problem, setting, res), open('pointing_res_%s.pkl' % (setting['label']), 'wb'))